<a href="https://colab.research.google.com/github/sinikoibra/GENAi/blob/main/Prompt_engg_web_scrp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#Libraries to install
!pip install -q langchain
!pip install -q langchain-openai
!pip install -q faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [18]:
tst_url =['https://www.paisabazaar.com/sbi-bank/sbi-elite-credit-card/']
from langchain_community.document_loaders import AsyncHtmlLoader#,AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer

# AsyncHtmlLoader used for concurrently loading HTML content from multiple URL
loader = AsyncHtmlLoader(tst_url)
docs = loader.load()

# We will use BeautifulSoupTransformer and specify which HTML tags to extract from the documents as we only need text from the HTML elements.
bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(
    docs, tags_to_extract=["span","p", "li", "div"]
)

Fetching pages: 100%|##########| 1/1 [00:01<00:00,  1.55s/it]


In [19]:
#building the model
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(openai_api_key="fill your Open AI API key",model="gpt-3.5-turbo",temperature=0.1)

Zero shot prompt(No Examples for LLM to refer to) + Simple prompt

In [ ]:
#We will use the PromptTemplate class from lanchain to build our prompt
from langchain.prompts.prompt import PromptTemplate
prompt_zer_sht=PromptTemplate(
        input_variables=["context", "question"],
        template = """Answer the following question based only on the provided context:

        <context>
        {context}
        </context>

        Question: {question}\n
        Answer:"""
    )

In [ ]:
#Run this block to see the final prompt that will be given to the LLM;question & context are the input variables that were given in the prompt template
prompt_zer_sht.format(
    question="Give a list of all the reward categories,corresponding points and required details for each category, also include the various benefits and corresponding details and include the fees from the given context",
    context=docs_transformed)

In [ ]:
#this is where we are building the chain i.e the output of current function is the input of next function.
chain_zer_sht=prompt_zer_sht|llm

In [ ]:
simple_question="""Give a list of all the reward categories,corresponding points and required details for each category,
also include the various benefits and corresponding details and include the fees from the given context"""

In [ ]:
output_zer_sht_simp=chain_zer_sht.invoke({"question":simple_question,
              "context":docs_transformed})

print(output_zer_sht_simp.content)

Reward Categories and Points:
1. Dining, Departmental stores, and Grocery Spends - 5X Reward Points
2. All other spends except fuel - 2 Reward Points per Rs. 100
3. International transactions - 2 Reward Points per Rs. 100

Milestone Rewards:
- 10,000 bonus rewards each on spending Rs. 3 Lakh and Rs. 4 Lakh in a year
- 15,000 bonus reward points on spending Rs. 5 Lakh in a year
- Additional 15,000 rewards on reaching annual spends of Rs. 8 Lakh

Benefits:
1. Welcome e-gift voucher worth Rs. 5,000
2. Free movie tickets worth Rs. 6,000 every year
3. Club Vistara Silver Membership
4. Trident Red Tier Membership
5. Dedicated assistance on Flower Delivery, Gift Delivery, Online Doctor Consultation
6. Low Foreign Currency Mark-up Charge of 1.99% on International usage
7. 1% fuel surcharge waiver on transactions between Rs. 500 to Rs. 4,000 at any petrol pump
8. Complimentary credit card fraud liability cover of Rs. 1 Lakh

Fees:
- Annual Fee: Rs. 4,999
- Renewal Fee: Rs. 4,999 (reversed on an

Zero shot prompt(No Examples for LLM to refer to) + detailed prompt

In [20]:
detailed_question="""
You're tasked with building a comprehensive database of credit card rewards. Your goal is to extract detailed information on SBI Elite Credit Card, including(if available) but not limited to:
Credit Card Name: Extract the name of each credit card.
Reward Categories: Identify the categories for which rewards are offered (e.g., dining, travel, groceries,).
Reward Points for Categories: Collect data on the number of reward points offered per tansaction amt for each category.
Reward points Terms and conditions:Gather specifics about the reward points system, including redemption options, point expiration policies, points cap.
Benefits Categories: Identify the categories for which benefits associated with each credit card (travel insurance, airport lounge access, concierge services, milestone benefits).
Details about Benefits: Provide in-depth information about each benefit, including coverage limits, eligibility criteria, and any additional fees, discounts, bonus points, voucher, airport lounge access.
Fees detail: Gather the joining fee,renewal fee,minimum spending for renewal fee waiveoff.
Your task is to compile this information into a structured dataset for analysis and comparison purposes. Be thorough in your extraction, ensuring accuracy and completeness of the data.Only use the below provided context for anwering the above question, incase you don't find certain answer don't make answers of your own.
"""

In [ ]:
output_zer_sht_dtail=chain_zer_sht.invoke({"question":detailed_question,
              "context":docs_transformed})
print(output_zer_sht_dtail.content)

Credit Card Name: SBI Elite Credit Card

Reward Categories: Dining, Departmental stores, Grocery Spends, International transactions

Reward Points for Categories:
- 5X Reward Points on Dining, Departmental stores, and Grocery Spends
- 2 Reward Points per Rs. 100 on all other spends, except fuel
- 2 Reward Points on every Rs.100 spent on International transactions

Reward points Terms and conditions:
- 4 Reward Points = Rupee 1
- Milestone Rewards: Up to 50,000 Bonus Reward Points worth Rs. 12,500 in a year

Benefits Categories: Rewards, Travel, Dining, Movies, Lounge Access, Club Vistara Membership, Trident Privilege Membership

Details about Benefits:
- Welcome e-gift voucher worth Rs. 5,000
- Free movie tickets worth Rs. 6,000 every year
- Club Vistara Silver Membership
- Trident Red Tier Membership
- Dedicated assistance on Flower Delivery, Gift Delivery, Online Doctor Consultation
- Low Foreign Currency Mark-up Charge of 1.99% on International usage
- 1% fuel surcharge waiver
- Com

In [23]:
examples=[
    {
"context": """Key Highlights of SBI Prime Credit Card
Best Suited or	Travel, shopping & rewards
Joining Fee	Rs. 2,999
Annual Fee	Rs. 2,999 (Waived off on spending Rs. 3 lakh in a year)
Welcome Benefit	E-gift vouchers worth Rs. 3,000 from top brands
Best Feature	10 reward points for every Rs. 100 spent on dining, groceries, departmental stores and movies
SBI Prime Credit Card lets you earn benefits on multiple categories, including dining, grocery, travel, golf, fuel and more. Below-mentioned are some of the major SBI Prime Credit

Card benefits:
Welcome Benefits worth Rs. 3,000
This credit card offers e-gift vouchers worth Rs. 3,000 of top brands as per the below details:

Bata/ Hush Puppies, Pantaloons, Shoppers Stop: Voucher can be redeemed by showing the code at the respective brand store
Yatra, Shoppers Stop: Voucher can be redeemed online at the time of purchase
Aditya Birla Retail Voucher: Redeemable at select brands, including Louis Philippe, Van Heusen, Allen Solly, Peter England, Planet Fashion, and Simon Carter

You can earn up to 20 reward points per transaction, which makes it one of the best rewards credit cards in the market. You can also pay your outstanding card bills via accumulated reward points. Here are the details:

20 reward points for every Rs. 100 spent on your birthday*
10 reward points for every Rs. 100 spent on dining, groceries, departmental stores and movies
2 reward points for every Rs. 100 spent on all other retail purchases, except for fuel
*Reward points earned on birthday spends (one day before, on, and one day after) are capped at 2,000 reward points per calendar year.
4 reward points = Re. 1 | 1 reward point = Re. 0.25

Multiple Milestone Benefits
On achieving the spending milestones, you can get complimentary vouchers and a renewal fee waiver via this credit card. Below mentioned are the details:

Pizza-hut gift voucher worth Rs. 1,000 on spending Rs. 50,000 in a calendar quarter
Renewal fee waiver on annual spends of Rs. 3 lakh
Yatra/ Pantaloons gift voucher worth Rs. 7,000 on achieving annual spends of Rs. 5 lakh

Travel Benefits
This credit card also offers complimentary Trident Privilege Red Tier Membership, Club Vistara Membership and international as well as domestic lounge visits for primary cardholders. With these memberships, you can avail exclusive discount and privileges as per the below details:

Complimentary Trident Privilege Red Tier Membership

1,000 welcome points on registration
1,500 bonus points on your first stay
Extra 1,000 hotel credit on an extended night stay (use promo code: SBITH)
10% instant discount on bookings via hotel website
10 points on every Rs. 100 spent during your stay at participating hotels

Complimentary Club Vistara Membership
Complimentary Club Vistara Silver Membership
1 upgrade voucher
9 Club Vistara (CV) Points for every Rs. 100 spent on Vistara flights

Complimentary Lounge Access
4 complimentary lounge visits per calendar year to international priority pass lounges, outside India (Max. 2 visits in a quarter)
8 complimentary lounge visits per calendar year to domestic lounges in India (Max. 2 visits per quarter)

Apart from the benefits mentioned above, this credit card also offers a fuel surcharge waiver and exclusive privileges for Visa cardholders. Below mentioned are some of the other SBI Card Prime benefits:

1% fuel surcharge waiver on your transactions between Rs. 500 and Rs. 4,000 across all petrol pumps in India
Up to 10% off on Hertz car rental
Up to 35% discount on Avis Car rentals
Exclusive benefits across 900 luxury hotels worldwide

Joining Fee	Rs. 2,999
Annual/ Renewal Fee	Rs. 2,999 (Waived off on spending Rs. 3 lakh in a year)""",
	"question": """You're tasked with building a comprehensive database of credit card rewards. Your goal is to extract detailed information on SBI prime credit card, including(if available) but not limited to:
Credit Card Name: Extract the name of each credit card.
Reward Categories: Identify the categories for which rewards are offered (e.g., dining, travel, groceries,).
Reward Points for Categories: Collect data on the number of reward points offered per tansaction amt for each category.
Reward points Terms and conditions:Gather specifics about the reward points system, including redemption options, point expiration policies, points cap.
Benefits Categories: Identify the categories for which benefits associated with each credit card (travel insurance, airport lounge access, concierge services, milestone benefits).
Details about Benefits: Provide in-depth information about each benefit, including coverage limits, eligibility criteria, and any additional fees, discounts, bonus points, voucher.
Fees detail: Gather the joining fee,renewal fee and minimum spending for renewal fee waiveoff
Your task is to compile this information into a structured dataset for analysis and comparison purposes. Be thorough in your extraction, ensuring accuracy and completeness of the data.Only use the below provided context for anwering the above question, incase you don't find certain answer don't make answers of your own.""",
	"answer": """Credit card name:SBI Prime Credit Card
Reward Categories:
-Dining
-groceries
-departmental stores
-movies
-all other retail purchases (excluding fuel)
Reward Points for Categories:
- 20 reward points for every Rs. 100 spent on birthday
- 10 reward points for every Rs. 100 spent on dining, groceries, departmental stores, and movies
- 2 reward points for every Rs. 100 spent on all other retail purchases (except fuel)
Reward points Terms and conditions:
- Point expiration policies: Reward points earned on birthday capped at 2,000 points per calendar year.
- 4 reward points = Re. 1 | 1 reward point = Re. 0.25
Benefits Categories:
-Travel
-shopping
-Welcome benefits
-Milestone Benefits
-Other Benefit
Details about Benefits:
-Milestone Benefits:
--Pizza-hut gift voucher on spending Rs. 50,000 in a calendar quarter
--Yatra/Pantaloons gift voucher on achieving annual spends of Rs. 5 lakh
- Welcome Benefits : e-gift vouchers worth Rs. 3,000 of top brands as per the below details:
--Bata/ Hush Puppies, Pantaloons, Shoppers Stop: Voucher can be redeemed by showing the code at the respective brand store
--Yatra, Shoppers Stop: Voucher can be redeemed online at the time of purchase
--Aditya Birla Retail Voucher: Redeemable at select brands, including Louis Philippe, Van Heusen, Allen Solly, Peter England, Planet Fashion, and Simon Carter
- Travel Benefits:
--offers complimentary Trident Privilege Red Tier Membership
--Club Vistara Membership
--International as well as domestic lounge visits for primary cardholders.4 complimentary lounge visits per calendar year to international priority pass lounges, outside India (Max. 2 visits in a quarter),8 complimentary lounge visits per calendar year to domestic lounges in India (Max. 2 visits per quarter)
Membership benefits:
-Trident Privilege Red Tier Membership:
--1,000 welcome points on registration
--1,500 bonus points on your first stay
--Extra 1,000 hotel credit on an extended night stay (use promo code: SBITH)
--10% instant discount on bookings via hotel website
--10 points on every Rs. 100 spent during your stay at participating hotels
-Complimentary Club Vistara Membership:
--Complimentary Club Vistara Silver Membership
--1 upgrade voucher
--9 Club Vistara (CV) Points for every Rs. 100 spent on Vistara flights
-Other Benefits:
--Fuel surcharge waiver
--exclusive privileges for Visa cardholders, discounts on car rentals
Fees:
-Joining Fee	Rs. 2,999
-Annual/ Renewal Fee	Rs. 2,999 (Waived off on spending Rs. 3 lakh in a year)"""
	},
	{
	"context": """Key Highlights of SBI SimplySAVE Credit Card
Best Suited For	Beginners | Rewards on Everyday Expenses
Joining Fee	Rs. 499
Annual Fee	Rs. 499 (Reversed on spending Rs. 1 lakh in a year)
Welcome Benefit	2,000 bonus reward points on spending Rs. 2,000 within 60 days of card issuance
Best Feature	10X reward points on dining, movies, grocery and departmental store spends
Paisabazaar’s Rating	★★★ (3/5)

Up to 10X Reward Points
This credit card lets you earn up to 10X reward points on your purchases at selected categories. Below mentioned are the details regarding the same:

10 reward points on every Rs. 150 spent on dining, movies, grocery and departmental store purchases
1 reward point per Rs. 150 spent on other categories

Other SBI SimplySAVE Credit Card Benefits
SBI SimplySAVE Credit Card also offers other benefits, such as bonus reward points, fuel surcharge waiver and annual fee waiver. Here are all the details regarding the same:

Welcome Benefit: 2,000 reward points on spending Rs. 2,000 within 60 days of card issuance.

Fuel Surcharge Waiver: 1% fuel surcharge waiver on your transactions between Rs. 500 to Rs. 3,000 at any petrol pump in India. You can avail a maximum waiver of Rs. 100 per statement cycle.

Annual Fee Reversal: Annual membership fee reversal from the second year on spending Rs. 1 lakh in the previous year""",
"question": """You're tasked with building a comprehensive database of credit card rewards. Your goal is to extract detailed information on SBI SimplySAVE Credit Card, including(if available) but not limited to:
Credit Card Name: Extract the name of each credit card.
Reward Categories: Identify the categories for which rewards are offered (e.g., dining, travel, groceries,).
Reward Points for Categories: Collect data on the number of reward points offered per tansaction amt for each category.
Reward points Terms and conditions:Gather specifics about the reward points system, including redemption options, point expiration policies, points cap.
Benefits Categories: Identify the categories for which benefits associated with each credit card (travel insurance, airport lounge access, concierge services, milestone benefits).
Details about Benefits: Provide in-depth information about each benefit, including coverage limits, eligibility criteria, and any additional fees, discounts, bonus points, voucher.
Fees detail: Gather the joining fee,renewal fee and minimum spending for renewal fee waiveoff
Your task is to compile this information into a structured dataset for analysis and comparison purposes. Be thorough in your extraction, ensuring accuracy and completeness of the data.Only use the below provided context for anwering the above question, incase you don't find certain answer don't make answers of your own.""",
"answer":"""Credit Card Name: SBI SimplySAVE Credit Card
Reward Categories: Dining, groceries, departmental stores, movies, all other retail purchases (excluding fuel)
Reward Points for Categories:
- 10 reward points for every Rs. 100 spent on dining, groceries, departmental stores, and movies
- 1 reward point per Rs. 150 spent on all other retail purchases (excluding fuel)
Benefits Categories:
-Welcome Benefit
-Fuel Surcharge Waiver
Details of Benefits:
-Welcome Benefit: 2,000 reward points on spending Rs. 2,000 within 60 days of card issuance.
-Fuel Surcharge Waiver: 1% fuel surcharge waiver on your transactions between Rs. 500 to Rs. 3,000 at any petrol pump in India. You can avail a maximum waiver of Rs. 100 per statement cycle.
Fees
-Joining Fee Rs. 499
-Annual/ Renewal Fee Rs. 499 (Waived off on spending Rs. 1 lakh in a year)"""
	}
	]

Few Shot Prompt(Examples for LLM to refer to) + simple prompt

In [21]:
#We will use the PromptTemplate class from lanchain to build our example prompt;The examples need to be sent into the LLM in structured form
from langchain.prompts.prompt import PromptTemplate
example_prompt=PromptTemplate(
        input_variables=["context", "question","answer"],
        template = """Answer the following question based only on the provided context:

        <context>
        {context}
        </context>

        Question: {question}\n
        Answer:{answer}
        """
    )

In [24]:
from langchain.prompts.few_shot import FewShotPromptTemplate

few_sht_simp_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""You're tasked with building a comprehensive database of credit card rewards.
    Your goal is to extract detailed information only from the given context and if you don't find certain answer don't make answers of your own.
    Below are some examples for your reference try to give the answers similar to the hierarchy in the examples""",
    suffix="""
            Question: {question}\n
                <context>
                {context}
                </context>\n


                Answer:
        """,
    input_variables=["question","context"],
    example_separator="\n\n"
)

In [ ]:
chain_few_sht=few_sht_simp_prompt|llm

In [ ]:
output_few_sht_simp=chain_few_sht.invoke({"question":simple_question,
              "context":docs_transformed})

print(output_few_sht_simp.content)

Reward Categories:
- Dining
- Departmental stores
- Grocery
- International transactions

Reward Points for Categories:
- 5X Reward Points on Dining, Departmental stores, and Grocery Spends
- 2 Reward Points per Rs. 100 on all other spends (except fuel)
- 2 Reward Points on every Rs. 100 spent on International transactions

Reward Points Terms and Conditions:
- 4 Reward Points = Re. 1
- Milestone Rewards:
  - 10,000 bonus rewards each on spending Rs. 3 Lakh and Rs. 4 Lakh in a year
  - 15,000 bonus reward points on spending Rs. 5 Lakh in a year
  - Additional 15,000 rewards on reaching annual spends of Rs. 8 Lakh

Benefits Categories:
- Welcome Benefit
- Milestone Rewards
- Free movie tickets
- Exclusive Travel Benefits
- Airport Lounge Access
- Other Benefits

Details of Benefits:
- Welcome Benefit: Rs. 5,000 worth welcome gift vouchers
- Free movie tickets worth Rs. 6,000 every year
- Exclusive Travel Benefits:
  - Club Vistara Silver Membership
  - Trident Red Tier Membership
- Airp

Few Shot Prompt + detailed prompt

In [ ]:
output_few_sht_dtail=chain_few_sht.invoke({"question":detailed_question,
              "context":docs_transformed})
print(output_few_sht_dtail.content)

Credit Card Name: SBI Elite Credit Card
Reward Categories: 
- Dining
- Departmental stores
- Grocery
- International transactions
Reward Points for Categories: 
- 5X Reward Points on Dining, Departmental stores, and Grocery Spends
- 2 Reward Points per Rs. 100 on all other spends (excluding fuel)
- 2 Reward Points on every Rs. 100 spent on International transactions
Reward points Terms and conditions:
- 4 Reward Points = Re. 1
- Point redemption options: Online, partner merchant store, or via phone
- Points can be redeemed in multiples of 2000 points
Benefits Categories: 
- Welcome Benefit
- Milestone Rewards
- Free movie tickets
- Exclusive Travel Benefits
- Airport Lounge Access
- Other Benefits
Details about Benefits: 
- Welcome Benefit: Rs. 5,000 worth welcome gift vouchers
- Milestone Rewards: Up to 50,000 Bonus Reward Points annually
- Free movie tickets worth Rs. 6,000 every year
- Exclusive Travel Benefits: Club Vistara and Trident Privilege Membership
- Airport Lounge Access: 

# Retrieval
Just like humans, too much information can overwhelm the LLM (Language Model), so we'll simplify it by breaking it into small pieces. Then, we'll look for the most likely set of pieces that contain the answer and send them as context to the LLM.

In [25]:
#We are splitting the files on the basis of
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
split_docs=text_splitter.split_documents(docs_transformed)

In [26]:
#Then we will convert the split files into vectors
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key="fill your Open AI API key",model="text-embedding-3-large")

In [27]:
 #We ingest documents into the vectorstore and create FAISS index
from langchain_community.vectorstores import FAISS
db_context = FAISS.from_documents(split_docs, embeddings)

We will use two ways to deal with the split files

---


Type 1: Stuff

Type 2: Refine


Let say we took 5 files out of the splited file set

-> Stuff type: we combine all 5 files and build one context file and feed it into the LLM

-> Refine type: we feed the files one by one and ask the LLM to refine the anwers(if required) on the basis of new information in the given context


In [28]:
#STUFF CHAIN TYPE

from langchain.prompts.few_shot import FewShotPromptTemplate

# below prompt is for stuff chain type hence it should mandatorily include following 2 input variables "summaries" and "question".
stuff_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""You're tasked with building a comprehensive database of credit card rewards.
    Your goal is to extract detailed information only from the given context and if you don't find certain answer don't make answers of your own.
    Below are some examples for your reference try to give the answers similar to the hierarchy in the examples""",
    suffix="""
            Question: {question}\n
                <context>
                {summaries}
                </context>\n


                Answer:
        """,
    input_variables=["question","summaries"],
    example_separator="\n\n"
)

In [29]:
from langchain.chains import RetrievalQAWithSourcesChain

#instead of building the chain manually we will use one of the prebuilt chains from langchain
stuff_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db_context.as_retriever(search_kwargs={"k":10}),#here the value K will determine within the split files set how many files should be used
     chain_type_kwargs={
         "verbose": False,
         "prompt": stuff_prompt,

     },
)

In [30]:
output_stuff_chain=stuff_chain.invoke({"question":detailed_question})
print(output_stuff_chain["answer"])
#output_stuff_chain

Credit Card Name: SBI Elite Credit Card
Reward Categories:
- Dining
- Departmental stores
- Grocery
- International transactions
- All other spends (excluding fuel)
Reward Points for Categories:
- 5X Reward Points on Dining, Departmental stores, and Grocery Spends
- 2 Reward Points per Rs. 100 on all other spends (excluding fuel)
- 2 Reward Points on every Rs. 100 spent on International transactions
Reward points Terms and conditions:
- Points can be redeemed in multiples of 2000 points unless otherwise stated
- Point expiration policies not specified
- Points cap not specified
Benefits Categories:
- Welcome Benefit
- Fuel Surcharge Waiver
- Travel Benefits
Details about Benefits:
- Welcome Benefit: Rs. 5,000 worth welcome gift vouchers
- Fuel Surcharge Waiver: 1% fuel surcharge waiver on transactions between Rs. 500 to Rs. 4,000 at any petrol pump in India
- Free movie tickets worth Rs. 6,000 every year
- Exclusive Travel Benefits: Club Vistara Silver Membership, including 1 complimen

In [ ]:
#REFINE CHAIN TYPE

from langchain.prompts.few_shot import FewShotPromptTemplate

# in refine chain type we need two types of prompt, the first prompt is question prompt, this is the prompt for the first file, as of now there is no existing answer to refine this will generate the first answer
question_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""You're tasked with building a comprehensive database of credit card rewards.
    Your goal is to extract detailed information only from the given context and if you don't find certain answer don't make answers of your own.
    Below are some examples for your reference try to give the answers similar to the hierarchy in the examples""",
    suffix="""
            Question: {question}\n
                <context>
                {context_str}
                </context>\n


                Answer:
        """,
    input_variables=["question","context_str"],
    example_separator="\n\n"
)

In [ ]:
#Refine prompt is the 2nd prompt this has "existing answer" as an input variable, from here on the answers will be refined on the basis of new context provided to the LLM
refine_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""You're tasked with building a comprehensive database of credit card rewards.
    Your goal is to extract detailed information only from the given context and if you don't find certain answer don't make answers of your own.
    Below are some examples for your reference try to give the answers similar to the hierarchy in the examples""",
    suffix="""
        The original question is as follows:Question: {question}\n
    We have provided an existing answer, including sources: {existing_answer}
    We have the opportunity to refine the existing answer(only if needed) with some more context below.

                <context>
                {context_str}
                </context>\n
    Given the new context, refine the original answer to better answer the question.
    If you do update it, please update the sources as well. If the context isn't useful, return the original answer.

                Answer:
        """,
    input_variables=["question","context_str","existing_answer"],
    example_separator="\n\n"
)

In [ ]:
Refine_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=db_context.as_retriever(search_kwargs={"k":10}),
     chain_type_kwargs={
         "verbose": False,
         "question_prompt": question_prompt,#both the prompts are given in the arguments
         "refine_prompt": refine_prompt,

     },
)

In [ ]:
output_Refine_chain=Refine_chain.invoke({"question":detailed_question})
print(output_Refine_chain["answer"])

Credit Card Name: SBI Elite Credit Card
Reward Categories: Dining, departmental stores, groceries, international transactions, hotel bookings, Vistara Airlines tickets
Reward Points for Categories: 
- 5X Reward Points on Dining, Departmental stores, and Grocery Spends
- 2 Reward Points per Rs. 100 on all other spends (except fuel)
- 2 Reward Points on every Rs. 100 spent on International transactions
- 10% discount on participating Trident hotels
- 2.25% rewards rate on Vistara Airlines tickets
Benefits Categories: Travel insurance, airport lounge access, concierge services, milestone benefits
Details about Benefits: 
- Free movie tickets worth Rs. 6,000 every year: Complimentary movie tickets worth Rs. 6,000 annually, valid for at least 2 tickets per booking with a maximum benefit of Rs. 250 per ticket. Convenience fee applicable.
- Exclusive Travel Benefits: Club Vistara Silver Membership with 1 complimentary Upgrade Voucher and 9 Club Vistara Points for every Rs. 100 spent on Vistar

# Similarity Comparision Using Cosine Similarity

In [ ]:
manual_output="""
Credit Card Name: SBI Elite Credit Card
Reward Categories:
- Dining
- Departmental stores
- Grocery
- International transactions
- All other spends (excluding fuel)
Reward Points for Categories:
- 5X Reward Points on Dining, Departmental stores, and Grocery Spends
- 2 Reward Points per Rs. 100 on all other spends (except fuel)
- 2 Reward Points on every Rs.100 spent on International transactions
Reward points Terms and conditions:
- 4 Reward Points = Rupee 1
- Points can be redeemed online, at partner merchant stores, or by calling the SBI Card helpline
Benefits Categories:
- Welcome Benefit
- Milestone Rewards
- Club Vistara and Trident Privilege Membership
- Airport Lounge Access
Details about Benefits:
- Welcome Benefit:
--Rs. 5,000 worth welcome gift vouchers
--choose the brand of your choice from the varied brands that include Hush Puppies/ Bata, Yatra.com, Shoppers Stop and  Pantaloons
- Milestone Rewards:
--Up to 50,000 Bonus Reward Points worth Rs. 12,500 in a year
--10,000 bonus rewards each on spending Rs. 3 Lakh and Rs. 4 Lakh in a year
--15,000 bonus reward points on spending Rs. 5 Lakh in a year
--Additional 15,000 rewards on reaching annual spends of Rs. 8 Lakh
- Club Vistara Silver Membership:
--1 complimentary Upgrade Voucher
--9 Club Vistara Points for every Rs. 100 spent on Vistara Flights
- Trident Red Tier Membership:
--1,000 welcome points on registration
--10% instant discount on booking through the hotel’s website
--10 points per Rs. 100 spent for booking room at participating hotels
--1,500 points on your first stay
--Additional 1,000 hotel credit on extended night stay (use promo code: SBITH)
- Airport Lounge Access:
--2 complimentary Domestic Airport Lounge visits every quarter in India
--Complimentary membership to the Priority Pass Program worth $99,6 complimentary Airport Lounge visits per calendar year, outside India ( max. 2 visits per quarter)
-other Benefit
--Dedicated assistance on Flower Delivery, Gift Delivery
--Online Doctor Consultation (Concierge services)
--Complimentary credit card fraud liability cover of Rs. 1 Lakh
--Low Foreign Currency Mark-up Charge of 1.99% on International usage
Fees detail:
- Joining Fee: Rs. 4,999
- Renewal Fee: Rs. 4,999 (reversed on annual spends of Rs. 10 Lakh
"""

In [ ]:
import openai
from openai import OpenAI

client = OpenAI(
  api_key='fill your Open AI API key')

#  Function to encode text into OpenAI embeddings
def encode_text_openai(text):
    response = client.embeddings.create(input=text, model="text-embedding-3-large")
    return response.data[0].embedding

# Encode documents
# doc1_embedding_openai = encode_text_openai(output_stuff_txt[0])
# doc2_embedding_openai = encode_text_openai(output_wthout_retrival_txt[0])
manual_scrap_emb = encode_text_openai(manual_output)

zer_sht_simp_emb = encode_text_openai(output_zer_sht_simp.content)
zer_sht_dtail_emb = encode_text_openai(output_zer_sht_dtail.content)
few_sht_simp_emb = encode_text_openai(output_few_sht_simp.content)
few_sht_dtail_emb = encode_text_openai(output_few_sht_dtail.content)
stuff_chain_emb = encode_text_openai(output_stuff_chain["answer"])
Refine_chain_emb = encode_text_openai(output_Refine_chain["answer"])



In [ ]:
manual_scrap_emb = encode_text_openai(manual_output)

In [ ]:
def similarity_func(vec_1,vec_2):
    vec_1_mag=np.linalg.norm(vec_1)
    vec_2_mag=np.linalg.norm(vec_2)
    output=(np.dot(vec_1, vec_2))/(vec_1_mag*vec_2_mag)
    return output

In [ ]:
import numpy as np
similarity_func(zer_sht_simp_emb,manual_scrap_emb)

0.7954153582874908

In [ ]:
similarity_func(zer_sht_dtail_emb,manual_scrap_emb)

0.9648887797068172

In [ ]:
similarity_func(few_sht_simp_emb,manual_scrap_emb)

0.8021759018131726

In [ ]:
similarity_func(few_sht_dtail_emb,manual_scrap_emb)

0.9724721246398547

In [ ]:
similarity_func(stuff_chain_emb,manual_scrap_emb)

0.9708412361639641

In [ ]:
similarity_func(Refine_chain_emb,manual_scrap_emb)

0.9558761840786402